<a href="https://colab.research.google.com/github/fachiny17/machine_learning/blob/main/tensorflow_course/my_food_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get and loadt the dataset